In [ ]:
from pathlib import Path
from warnings import warn

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import RegularGridInterpolator
from tqdm.auto import tqdm

from bounded_rand_walkers.cpp import (
    bound_map,
    funky,
    generate_data,
    get_binned_2D,
    get_binned_data,
    get_cached_filename,
)
from bounded_rand_walkers.rad_interp import exact_radii_interp, inv_exact_radii_interp
from bounded_rand_walkers.relief_matrix_shaper import gen_shaper2D
from bounded_rand_walkers.rotation_steps import get_pdf_transform_shaper
from bounded_rand_walkers.shaper_general import gen_rad_shaper_exact, shaper_map
from bounded_rand_walkers.utils import (
    approx_edges,
    cache_dir,
    get_centres,
    match_ref,
    normalise,
)

mpl.rc_file("matplotlibrc")
Path("plots").mkdir(exist_ok=True)

In [ ]:
# Generate a single set of data.

pdf_kwargs = dict(width=2.0)


def get_f_i(r):
    """Calculate f_i for given radii."""
    return np.array([funky([c, 0], **pdf_kwargs) for c in r])


bound_name = "square"
n_bins = 300

data_kwargs = dict(
    cache_dir=cache_dir,
    samples=int(1e7),
    seed=np.arange(10),
    blocks=2,
    bound_name=bound_name,
    **pdf_kwargs
)

In [ ]:
filenames = get_cached_filename(squeeze=False, **data_kwargs)
if not all(p.is_file() for p in filenames):
    generate_data(squeeze=False, max_workers=2, cache_only=True, **data_kwargs)

(
    g_x_edges,
    g_y_edges,
    g_x_centres,
    g_y_centres,
    f_t_x_edges,
    f_t_y_edges,
    f_t_x_centres,
    f_t_y_centres,
    f_t_r_edges,
    f_t_r_centres,
    g_numerical,
    f_t_numerical,
    f_t_r_numerical,
) = get_binned_data(filenames=filenames, n_bins=n_bins)

f_i_r_analytical = get_f_i(f_t_r_centres)

In [ ]:
# 1D shaper calculation.
r_shaper = gen_rad_shaper_exact(f_t_r_centres, vertices=bound_name)

# Select valid elements.
valid_r = ~np.isclose(r_shaper, 0)
f_i_r_num_radii = f_t_r_centres[valid_r].copy()

# 1D reconstruction of the intrinsic pdf as a function of step length only.
f_i_r_num = f_t_r_numerical[valid_r] / r_shaper[valid_r]

# 1D analytical transformed distribution.
f_t_r_analytical = f_i_r_analytical * r_shaper

# Analytical transformed.
f_i_r_analytical_trans = f_t_r_analytical[valid_r] / r_shaper[valid_r]

In [ ]:
# Generate a multiple sets of data in order to visualise the uncertainty.

sub_data_kwargs = data_kwargs.copy()
sub_exp = 5
sub_data_kwargs["samples"] = int(10 ** sub_exp)
sub_data_kwargs["seed"] = np.arange(100)

sub_filenames = get_cached_filename(squeeze=False, **sub_data_kwargs)
if not all(p.is_file() for p in sub_filenames):
    generate_data(squeeze=False, max_workers=2, cache_only=True, **sub_data_kwargs)

In [ ]:
# Manually bin the step sizes radially to compute minimum and maximum bounds.
ignore_initial = 1000

f_t_r_num_all = np.empty(
    (len(sub_data_kwargs["seed"]), f_t_r_centres.size), dtype=np.float64
)

for i, filename in enumerate(
    tqdm(sub_filenames, desc="Binning data radially", smoothing=0)
):
    if filename is not None and filename.is_file():
        # Cached data exists.
        saved = np.load(filename)
        positions = saved["positions"][ignore_initial:]
        steps = saved["steps"][ignore_initial:]
        saved.close()
    else:
        warn(f"Filename '{filename}' was not found.")
        continue

    # Radially bin the step size data.
    f_t_r_num_all[i] = np.histogram(np.linalg.norm(steps, axis=1), bins=f_t_r_edges)[0]

In [ ]:
# Normalise the binned data generated above.
f_t_r_length = np.abs(f_t_r_edges[1] - f_t_r_edges[0])
f_t_r_num_all /= np.sum(f_t_r_num_all * f_t_r_length, axis=1).reshape(-1, 1)

f_t_r_num_mins = np.min(f_t_r_num_all, axis=0)
f_t_r_num_maxs = np.max(f_t_r_num_all, axis=0)
f_t_r_num_stds = np.std(f_t_r_num_all, axis=0)
f_t_r_num_mean = np.mean(f_t_r_num_all, axis=0)

In [ ]:
# 1D reconstruction of the intrinsic pdf as a function of step length only.
f_i_r_all_list = []
for i in range(f_t_r_num_all.shape[0]):
    f_i_r_all_list.append((f_t_r_num_all[i][valid_r] / r_shaper[valid_r])[None])
f_i_r_num_all = np.vstack(f_i_r_all_list)

f_i_r_num_mins = np.min(f_i_r_num_all, axis=0)
f_i_r_num_maxs = np.max(f_i_r_num_all, axis=0)
f_i_r_num_stds = np.std(f_i_r_num_all, axis=0)
f_i_r_num_mean = np.mean(f_i_r_num_all, axis=0)

In [ ]:
fewer_samples_label = fr"$10^{sub_exp}$ Samples"

# Plot f_t and f_i.
fig, axes = plt.subplots(1, 2, figsize=(6.3, 2.52), sharey=False)

# Plot f_t.
ax = axes[0]

ax.plot(
    f_t_r_centres,
    normalise(f_t_r_edges, f_t_r_analytical * f_t_r_centres),
    label="Analytical",
    zorder=1,
)
ax.plot(
    f_t_r_centres,
    normalise(f_t_r_edges, f_t_r_numerical),
    label="Numerical",
    zorder=2,
    linestyle="--",
    c="C1",
)

f_t_factor = normalise(f_t_r_edges, f_t_r_num_mean, return_factor=True)
ax.fill_between(
    f_t_r_centres,
    f_t_factor * (f_t_r_num_mean - f_t_r_num_stds),
    f_t_factor * (f_t_r_num_mean + f_t_r_num_stds),
    alpha=0.5,
    zorder=1,
    color="C4",
    label=fewer_samples_label,
)

# Plot f_i.
ax = axes[1]

f_i_r_analytical_trans_norm = normalise(
    f_i_r_num_radii, f_i_r_analytical_trans * f_i_r_num_radii
)

(t1,) = ax.plot(
    f_i_r_num_radii,
    f_i_r_analytical_trans_norm,
    # label="Transformed Analytical",
    zorder=2,
    linestyle="--",
    c="C0",
)

(t2,) = ax.plot(
    f_t_r_centres,
    match_ref(
        x=f_t_r_centres,
        y=f_i_r_analytical * f_t_r_centres,
        ref_x=f_i_r_num_radii,
        ref_y=f_i_r_analytical_trans_norm,
    ),
    # label="Analytical",
    zorder=1,
    linestyle="-.",
    c="C2",
)

(b1,) = ax.plot(
    f_i_r_num_radii,
    normalise(f_i_r_num_radii, f_i_r_num),
    # label="Numerical",
    zorder=1,
    c="C1",
    linestyle="--",
)

# Fix y-axis limits so they do not blow up due to the variability of the below.
ylim = ax.get_ylim()
ax.set_ylim(*ylim)
ax.autoscale(False)

f_i_factor = normalise(f_i_r_num_radii, f_i_r_num_mean, return_factor=True)
b2 = ax.fill_between(
    f_i_r_num_radii,
    f_i_factor * (f_i_r_num_mean - f_i_r_num_stds),
    f_i_factor * (f_i_r_num_mean + f_i_r_num_stds),
    alpha=0.5,
    zorder=1,
    color="C4",
    # label='Fewer Samples'
)

# Labels.
axes[0].set_xlabel(r"$\ell$")
axes[0].set_ylabel(r"$\tilde{f}_t(\ell)$")

axes[1].set_xlabel(r"$\ell$")
axes[1].set_ylabel(r"$\tilde{f}_i(\ell)$", labelpad=-18)

# Legends.
axes[0].legend()

l1 = axes[1].legend([t1, t2], ["Transf. Analytical", "Analytical"], loc="upper right")
l2 = axes[1].legend([b1, b2], ["Numerical", fewer_samples_label], loc="upper left")
axes[1].add_artist(l1)

# Grids and titles.
for ax, title in zip(axes, ["(a)", "(b)"]):
    ax.text(0, 1.04, title, transform=ax.transAxes)

for ax in axes:
    ax.set_ylim(-0.05, 1.18)

axes[1].set_yticks(np.arange(0, 1.2, 0.2))
axes[1].set_yticklabels(["0.0", "", "", "", "", "1.0"])

# Move the two subplots closer to each other.
fig.tight_layout()

# Finally, save into the 'plots' directory.
fig.savefig((Path("plots") / "square_reconstruction_old").with_suffix(".pdf"))

In [ ]:
analytical_f_i_args = (
    f_t_r_centres,
    match_ref(
        x=f_t_r_centres,
        y=f_i_r_analytical * f_t_r_centres,
        ref_x=f_i_r_num_radii,
        ref_y=f_i_r_analytical_trans_norm,
    ),
)

analytical_f_i_kwargs = dict(
    label=r"Analytical $\tilde{f}_i(\ell)$",
    zorder=1,
    linestyle="-.",
    c="C2",
)

# Plot f_t and f_i.
fig, axes = plt.subplots(1, 2, figsize=(6.3, 2.52), sharey=False)

# Plot f_t.
ax = axes[0]

ax.plot(
    f_t_r_centres,
    normalise(f_t_r_edges, f_t_r_analytical * f_t_r_centres),
    label=r"Analytical $\tilde{f}_t(\ell)$",
    zorder=1,
)

f_t_factor = normalise(f_t_r_edges, f_t_r_num_mean, return_factor=True)
ax.fill_between(
    f_t_r_centres,
    f_t_factor * (f_t_r_num_mean - f_t_r_num_stds),
    f_t_factor * (f_t_r_num_mean + f_t_r_num_stds),
    alpha=0.5,
    zorder=1,
    color="C4",
    label=r"Numerical $\tilde{f}_t(\ell)$",
)
# Plot this for comparison.
ax.plot(
    *analytical_f_i_args,
    **analytical_f_i_kwargs,
)

# Plot f_i.
ax = axes[1]

ax.plot(
    *analytical_f_i_args,
    **analytical_f_i_kwargs,
)

f_i_factor = normalise(f_i_r_num_radii, f_i_r_num_mean, return_factor=True)
ax.fill_between(
    f_i_r_num_radii,
    f_i_factor * (f_i_r_num_mean - f_i_r_num_stds),
    f_i_factor * (f_i_r_num_mean + f_i_r_num_stds),
    alpha=0.5,
    zorder=1,
    color="C4",
    label=r"Numerical $\tilde{f}_i(\ell)$",
)

# Labels.
for ax in axes:
    ax.set_xlabel(r"$\ell$")

ylabel = "Step Probability    "
ylabel_fs = mpl.rcParams["axes.labelsize"] - 1.5
axes[0].set_ylabel(ylabel, fontsize=ylabel_fs)
axes[1].set_ylabel(ylabel, labelpad=-12, fontsize=ylabel_fs)

# Legends.
for ax in axes:
    ax.legend()

# Grids and titles.
for ax, title in zip(axes, ["(a)", "(b)"]):
    ax.text(0, 1.04, title, transform=ax.transAxes)

for ax in axes:
    ax.set_ylim(-0.05, 1.18)

axes[1].set_yticks(np.arange(0, 1.2, 0.2))
axes[1].set_yticklabels(["0.0", "", "", "", "", "1.0"])

# Move the two subplots closer to each other.
fig.tight_layout()

# Finally, save into the 'plots' directory.
fig.savefig((Path("plots") / "square_reconstruction").with_suffix(".pdf"))